# Trabalho Prático 3

Trabalho realizado pelo grupo 11:

* Beatriz Fernandes Oliveira, PG50942
    
* Bruno Filipe Machado Jardim, PG49997



## Algoritmo BIKE

Nesta segunda parte do trabalho prático 3, foi\-nos pedido:

1. A criação de um protótipo em Sagemath para o algoritmo BIKE, que se baseia no problema da descodificação de códigos lineares de baixa densidade que são simples de implementar.

2. E, posteriormente, para essa técnica pretende\-se implementar um KEM, que seja IND\-CPA seguro, e um PKE que seja IND\-CCA seguro.



In [14]:
import random
from cryptography.hazmat.primitives import hashes
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

In [50]:
class BIKE_KEM(object):
    
    def __init__(self, r, N, T, L):
        self.r = r
        self.n = N
        self.t = T
        self.l = L
        self.q = GF(2) # corpo finito de tamanho 2
        F.<x> = PolynomialRing(self.q)
        R.<x> = QuotientRing(F, F.ideal(x^self.r + 1))
        self.R = R

    def K(self, e0, e1):
        digest = hashes.Hash(hashes.SHA256())
        digest.update(e0.encode())
        digest.update(e1.encode())
        return digest.finalize()
    
    def Hamming(self, x):
        return sum([1 if a == self.q(1) else 0 for a in x])
    
    def ppairwise(self, x1, x2):
        return x1.pairwise_product(x2)
    
    def geraCoefs(self, sparse=3):
        coefs = [1]*sparse + [0]*(self.r - 2 - sparse)
        random.shuffle(coefs)
        return self.R([1] + coefs + [1])
    
    def geraError(self):
        err = [1]*self.t + [0]*(self.n - self.t)
        random.shuffle(err)
        return self.R(err[:self.r]),self.R(err[self.r:])
       
    def keyGenerator(self):
        h0 = self.geraCoefs()
        h1 = self.geraCoefs()
        g = self.geraCoefs()
        f = (h1 + g*h0, g)
        return (h0, h1), f
        
    def encaps(self, public):
        e = self.geraCoefs(self.t)
        e0, e1 = self.geraError()
        key = self.K(str(e0), str(e1))
        c = (e + e1 * public[0], e0 + e1 * public[1])
        return key, c
    
    def to_Vector_r(self, f):
        fl = f.list()
        V = VectorSpace(self.q, self.r)
        return V(fl + [0]*(self.r - len(fl)))
    
    def to_Vector_n(self, c):
        f = self.to_Vector_r(c[0]).list() + self.to_Vector_r(c[1]).list()
        V = VectorSpace(self.q, self.n)
        return V(f)
    
    def rotate_unit(self, unit):
        V = VectorSpace(self.q, self.r)
        v = V()
        v[0] = unit[-1]
        for i in range(self.r-1):
            v[i+1] = unit[i]
        return v
    
    def rotate(self, h):
        matrix = Matrix(self.q, self.r, self.r)
        matrix[0] = self.to_Vector_r(h)
        for i in range(1, self.r):
            matrix[i] = self.rotate_unit(matrix[i-1])
        return matrix
        
    def bitFlip(self, matrix, c, s):
        c_ = c
        s_ = s
        nItr = self.r
        while self.Hamming(s_) > 0 and nItr > 0:
            nItr -= 1
            pesos = [self.Hamming(s_.self.ppairwise(matrix[i])) for i in range(self.n)]
            maximo = max(pesos)
            for j in range(self.n):
                if pesos[j] == maximo:
                    c_[j] = 1 - c_[j]
                    z += matrix[j]
                    
        if nItr == 0:
            return None
        return c_
    
    def decaps(self, public, c):
        public_rotate = (self.rotate(public[0]), self.rotate(public[1]))
        H_matrix = block_matrix(2,1, [public_rotate[0], public_rotate[1]])
        c_vector = self.to_Vector_n(c)
        syndrome = c_vector * private[0]
        error = self.bitFlip(H_matrix, c_vector, syndrome)
        if error == None:
            print("Máximo de iterações atingidas")
            return None
        else:
            elist = error.list()
            error0, error1 = self.R(elist[:self.r]), self.R(elist[self.r:])
            e0, e1 = c[0] - error0, c[1] - error1
            if self.Hamming(self.to_Vector_r(e0)) + self.Hamming(self.to_Vector_r(e1)) != self.t:
                print("Erro")
                return None
        return self.K(str(e0), str(e1))

### Exemplo



In [51]:
bike = BIKE_KEM(257, 514, 16, 256) #r, n, t, l

private, public = bike.keyGenerator()

key_encaps, c = bike.encaps(public)

key_decaps = bike.decaps(private, c)

if key_encaps == key_decaps and key_decaps != None:
    print("A chave desencapsulada é igual à chave encapsulado. O algoritmo BIKE como um KEM funciona!")

Erro


## Implementação do PKE



In [5]:
class BIKE_PKE(object):
    
    def __init__(self, R, N, T, L):
        self.kem = BIKE_KEM(R,N,T,L)
        
    def keyGen(self):
        self.private, self.public = self.kem.keyGenerator()
        return self.private, self.public
    

### Exemplo



In [6]:
bike = BIKE_PKE(257, 514, 16, 256) #r, n, t, l

mensagem = "EC-TP3 Implementação do algoritmo BIKE com um PKE"